In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib                       # python3.4+ depreciated imp.reload. Use importlib.reload
import atus_var_dictionary as avd      # my way to get a grip of col/var meanings
importlib.reload(avd)                  # comment out when no longer making changes to atus_var_dictionary
%matplotlib inline

In [2]:
# file paths to initial csv files
data_dir= '/Users/twaddy/NOTES/thinkful/DataSciBootCampPrep/cvs_data/atus/'

sum_fp  = os.path.join(data_dir, 'atussum.csv')  #0.2 Gb
act_fp  = os.path.join(data_dir, 'atusact.csv')  #0.4 Gb
resp_fp = os.path.join(data_dir, 'atusresp.csv') #0.1 Gb
rost_fp = os.path.join(data_dir, 'atusrost.csv')
cps_fp  = os.path.join(data_dir, 'atuscps.csv')  #0.6 Gb
who_fp  = os.path.join(data_dir, 'atuswho.csv')

# file paths to intermediate csv files
travel_sum_fp   = os.path.join(data_dir, 'tw_atus_travel_sum.csv')
travel_act_fp   = os.path.join(data_dir, 'tw_atus_travel_act.csv')
travel_cps_fp   = os.path.join(data_dir, 'tw_atus_travel_cps.csv')
travel_resp_fp  = os.path.join(data_dir, 'tw_atus_travel_resp.csv')
travel_all_fp   = os.path.join(data_dir, 'tw_atus_travel_all.csv')

In [3]:
# As data files were large, these flags determine whether:
#     1) to read original cvs file, prune un-needed data and save intermediate to disk
#     2) to read the pruned intermediate cvs from disk
force_prune_activities_csv = False
force_prune_cps_csv = False
force_prune_resp_csv = False

In [5]:
if (not os.path.exists(travel_act_fp)) or force_prune_activities_csv:
    df_act  = pd.read_csv(act_fp)
    # Filter rows to retain activities designated as "travel", for *ANY* reason. (coded 6 digits starting with 18)
    df_travel_act = df_act.where(df_act.trcodep.map(lambda x: x if (str(x).startswith('18')) else np.nan)).dropna()

    # Filter columns of un-need data
    keeper_act_cols = ['tucaseid', 'tuactivity_n', 'tucc5', 'tuactdur24', 'tucumdur', 'trcodep', 'tewhere', 'tufinlwgt']
    df_travel_act = df_travel_act.filter(items=keeper_act_cols)

    print("Filtered to retain *only* travel activities:")
    print("#rows    reduced from {0} to {1}".format(df_act.shape[0],df_travel_act.shape[0]))
    print("#columns reduced from {0} to {1}".format(df_act.shape[1],df_travel_act.shape[1]))
    print("writing out a csv file for *just* travel activities...")
    df_travel_act.to_csv(path_or_buf=travel_act_fp, mode = 'w', index=False)
else:
    df_travel_act = pd.read_csv(travel_act_fp)

Filtered to retain *only* travel activities:
#rows    reduced from 3347093 to 675992
#columns reduced from 29 to 7
writing out a csv file for *just* travel activities...


In [6]:
# atuscps.csv file.  (CPS is the Current Population Survey. It's 625Mb)
if (not os.path.exists(travel_cps_fp)) or force_prune_cps_csv:
    df_cps  = pd.read_csv(cps_fp)
    keep_cps_col_linkers = ['tucaseid', 'tulineno']
    keep_cps_col_geog    = ['gestfips', 'gemetsta']         # gestfips = state
    keep_cps_occupation  = ['prdtocc1']                     # trdtocc1 = 22 = work in transportation
    keep_cps_school      = ['peschenr']                     # peschenr = 1 = enrolled in school
    keep_cps_education   = ['peeduca']

    all_cps_keepers  =  keep_cps_col_linkers +                        keep_cps_col_geog +                        keep_cps_occupation +                        keep_cps_education
    df_pruned_cps = df_cps.filter(items=all_cps_keepers)
    df_pruned_cps.to_csv(path_or_buf=travel_cps_fp, mode = 'w', index=False)
    print("#cps rows reduced from {0} to {1}".format(df_cps.shape[0],df_pruned_cps.shape[0]))
    print("#cps cols reduced from {0} to {1}".format(df_cps.shape[1],df_pruned_cps.shape[1]))
    print("writing out a csv file for cps data to help isolate, school run, commute and students...")
else:
    df_cps = pd.read_csv(travel_cps_fp)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#cps rows reduced from 942073 to 942073
#cps cols reduced from 238 to 6
writing out a csv file for cps data to help isolate, school run, commute and students...


In [8]:
if (not os.path.exists(travel_resp_fp)) or force_prune_resp_csv:
    df_resp  = pd.read_csv(resp_fp)
    keep_resp_cols_linkers     = ['tucaseid', 'tulineno', 'tufinlwgt']               #for join/merge
    keep_resp_temporal         = ['tudiaryday', 'tudiarydate', 'trholiday']          #for weekday/holiday exclusion
    keep_resp_cols_occ_ind_emp = ['trdtind1', 'trdtocc1', 'telfs', 'tespempnot']     #for taxi/bus driver exclusion
    keep_resp_cols_par_sch     = ['trnumhou', 'tryhhchild', 'trhhchild']             #for parent school run inclusion
    keep_resp_are_student      = ['teschenr', 'teschft']                        #for resp = school student inclusion

    all_resp_keepers =  keep_resp_cols_linkers +\
                        keep_resp_temporal +\
                        keep_resp_cols_occ_ind_emp +\
                        keep_resp_cols_par_sch +\
                        keep_resp_are_student

    df_pruned_resp = df_resp.filter(items=all_resp_keepers)
    df_pruned_resp.to_csv(path_or_buf=travel_resp_fp, mode = 'w', index=False)
    print("#cps rows reduced from {0} to {1}".format(df_resp.shape[0], df_pruned_resp.shape[0]))
    print("#cps cols reduced from {0} to {1}".format(df_resp.shape[1], df_pruned_resp.shape[1]))
    print("writing out a csv file for resp data")
else:
    df_resp = pd.read_csv(travel_resp_fp)


#cps rows reduced from 170842 to 170842
#cps cols reduced from 132 to 14
writing out a csv file for resp data


In [9]:
# Note that the activities dataframe (travel_act_df) still includes travel on weekends and holidays.
# The diarydate and whether a holiday is not in this activities data.
# It's recorded in the atusresp.csv
# We must now merge the "respondent" data.

In [10]:
# Before joining/merging, which columns in common in differnt csv files?
def get_cols_in_common(df1, df2):
    """Not very pandesque! Is there already a df method for this?"""
    df1_cols = list(df1.columns.values)
    df2_cols = list(df2.columns.values)
    overlap = list(set(df1_cols).intersection(df2))
    return overlap
print(get_cols_in_common(df_travel_act, df_resp))

['tucaseid']


In [18]:
df_merged_act_resp = pd.merge(df_travel_act, df_resp, on=['tucaseid'], how='inner')
print(df_merged_act_resp.shape)

(675992, 138)


In [20]:
# NB. weekdays are encoded such that  (1,2,3 -> 7)  == (Sun, Mon, Tue -> Sat)
# Query for: non-sundays AND non-saturdays AND non-holidays
weekday_travel_df = df_merged_act_resp.query('(tudiaryday != 7) & (tudiaryday != 1) & (trholiday == 0)').dropna()

In [21]:
# Reading the remaining needed data
#df_sum  = pd.read_csv(sum_fp)
df_rost = pd.read_csv(rost_fp)
df_who  = pd.read_csv(who_fp)

In [22]:
# 3 population isolation efforts are needed and will follow below:
#         1) Those parent respondents driving their student child to school.
#         2) Those parent respondents, without child drop-off, commuting to work, who don't work in transport.
#         3) Those high-schoolers/student respondents who are self-driving to/from school/college.

In [24]:
#Merging.  Roster and Who  ... then that result into the above "weekday_travel_act_resp_df"
print(get_cols_in_common(df_rost, df_who))
df_merged_rost_who = pd.merge(df_rost, df_who, on=['tucaseid', 'tulineno'], how='inner')
print(df_merged_rost_who.shape)

print(get_cols_in_common(weekday_travel_df, df_merged_rost_who))
df_merged_all = pd.merge(weekday_travel_df, df_merged_rost_who, on=['tucaseid', 'tulineno', 'tuactivity_n'], how='outer')
df_merged_all.to_csv(path_or_buf=travel_all_fp, mode = 'w', index=False)
df_merged_all.shape

['tucaseid', 'tulineno']
(2955410, 8)
['tucaseid', 'tulineno', 'tuactivity_n']


(3100642, 143)

In [ ]:
#From here, switching to another notebook: DS_Prep_Travel_Capstone_post_merge_analysis

3 population isolation efforts are needed and will follow below:  
        1) Those parent respondents driving under 13 student to school.  
        2) Those parent respondents, without child drop-off, commuting to work, who don't work in transport.  
        3) Those high-schoolers/student respondents who are self-driving to/from school/college.

In [ ]:
# respondent accompanied by under 13
print(weekday_travel_df.shape)
weekday_travel_with_u13 = df_merged_all.where(weekday_travel_df['tucc5'] > 0).dropna()
print(weekday_travel_with_u13.shape)
#weekday_travel_with_u13.head(2)

(355306, 138)


In [ ]:
#df_sum['all_travel'] = df_sum[['t180601', 't180699']].sum(axis=1)
#all_edu_travel_df  = df_sum[['t180601', 't180699']]

In [ ]:
school_travel_w_u13 = weekday_travel_with_u13.query('(trcodep == 180601) | (trcodep == 180699)').dropna()
print(school_travel_w_u13.shape)
#school_travel_w_u13.tuactdur24.plot(kind='hist')
# I'll return to this df when I have the respodent who are also self driving students

In [ ]:
weekday_travel_df.teage.plot(kind='hist')

In [ ]:
def average_minutes(data, activity_code):
    activity_col = "t{}".format(activity_code)
    data = data[['TUFINLWGT', activity_col]]
    data = data.rename(columns={"TUFINLWGT": "weight", activity_col: "minutes"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

def average_minutes2(data, activity_code):
    cols = activity_columns(data, activity_code)
    activity_data = data[cols]
    activity_sums = activity_data.sum(axis=1)
    data = data[['TUFINLWGT']]
    data['minutes'] = activity_sums
    data = data.rename(columns={"TUFINLWGT": "weight"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [ ]:
fig = plt.figure(figsize=(16, 5))
ax1 = fig.add_subplot(1, 1, 1)            # Create first  matplotlib axes
travel_with_under_13_df['tuactdur24'].plot(kind='hist',  ax = ax1, bins=90)
plt.title('Travel with a child under 13.')    
#plt.xlabel("Time spent travelling (mins)\nAverage = {0:.0f} (some don't travel at all)".format(travel_mean))
plt.xlabel("Time spent travelling (mins)")
plt.ylabel('Frequency of respondents travelling this time.')
plt.xlim([0, 90])
ymin, ymax = plt.ylim()
#plt.xticks(range(0, 600, 60))
#plt.vlines(travel_mean, 0, ymax, color='red')

In [ ]:
#df_rost.groupby('tucaseid').where(df_rost['tulineno'].max > 1)
# terrp (relationship) corresponds 22, 23, 27, 40 = own, grand, foster, non-hh yet own child

def has_child(x):
    if x in [22,23,27,40]:
        return 1
    else:
        return 0

#hh_non_solo = df_rost.groupby('tucaseid')['tulineno'].max()
#hh_with_child = df_rost.groupby('tucaseid').apply(has_child(df_rost['terrp']))                                                 
#hh_with_child

In [ ]:
# Summing across columns ( hence: axis = 1) all those named in the list: travel_18s
df_sum['all_travel'] = df_sum[travel_18s].sum(axis=1)
all_edu_travel_df  = df_sum[['t180601', 't180699']]


# The average time spent traveling by Americans:
all_travelling_any_reason = df_sum['all_travel'].sum()
all_travellers = df_sum['all_travel'].count()
mean_travel_any_reason = round(all_travelling_any_reason / all_travellers)
in_hours = mean_travel_any_reason/60
print("\nAverage time Americans spend traveling = {0:.0f} mins".format(mean_travel_any_reason))
print("(ie {0:.2f} hours ... Travelling for any reason.)\n".format(in_hours))

In [ ]:
work_travel_codes = ['t180501', 't180502', 't180589']
edu_travel_codes = ['t180601', 't180699']

all_work_travel_df = df_sum[['t180501', 't180502', 't180589']]
all_edu_travel_df  = df_sum[['t180601', 't180699']]

all_edu_travel_df = all_edu_travel_df.rename(columns={"t180601": "taking_class", 
                                                      't180699': "edu_n_e_c"})

In [ ]:
#act_rost_reports = pd.merge(activities_count, r_counts, on=['TESEX', 'TEAGE'], how='outer')
#act_rost_reports['reports_per_resp'] = act_rost_reports.reports / act_rost_reports.resp_count
#act_rost_reports.drop('resp_count', 1, inplace=True)
#act_rost_reports = act_rost_reports.groupby(['TUTIER1CODE', 'TUTIER2CODE', 'TUTIER3CODE', 'TESEX', 'TEAGE']).sum()
#act_rost_reports.head()